In [3]:
import pandas as pd

file_path = "../data/raw/raw_data.json"
df = pd.read_json(file_path, lines=True)

df.head()

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,"1616 Chapala St, Ste 2",{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA,NaN,NaN
1,87 Grasso Plaza Shopping Center,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...",Affton,1,38.551126,-90.335695,The UPS Store,63123,15,3.0,MO,mpf3x-BjTdTEA3yCZrAYPw,"{'Friday': '8:0-18:30', 'Monday': '0:0-0:0', '..."
2,5255 E Broadway Blvd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...",Tucson,0,32.223236,-110.880452,Target,85711,22,3.5,AZ,tUFrWirKiKi_TAnsVWINQQ,"{'Friday': '8:0-23:0', 'Monday': '8:0-22:0', '..."
3,935 Race St,"{'Alcohol': 'u'none'', 'BikeParking': 'True', ...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,1,39.955505,-75.155564,St Honore Pastries,19107,80,4.0,PA,NaN,"{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '..."
4,101 Walnut St,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Brewpubs, Breweries, Food",Green Lane,1,40.338183,-75.471659,Perkiomen Valley Brewery,18054,13,4.5,PA,mWMc6_wTdE0EUBKIGXDVfA,"{'Friday': '12:0-22:0', 'Saturday': '12:0-22:0..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225572 entries, 0 to 225571
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   address       225572 non-null  object 
 1   attributes    204971 non-null  object 
 2   categories    225415 non-null  object 
 3   city          225572 non-null  object 
 4   is_open       225572 non-null  int64  
 5   latitude      225572 non-null  float64
 6   longitude     225572 non-null  float64
 7   name          225572 non-null  object 
 8   postal_code   225572 non-null  object 
 9   review_count  225572 non-null  int64  
 10  stars         225572 non-null  float64
 11  state         225572 non-null  object 
 12  business_id   158001 non-null  object 
 13  hours         190686 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 24.1+ MB


In [5]:
df.isna().sum()

address             0
attributes      20601
categories        157
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     67571
hours           34886
dtype: int64

In [7]:
import json

# Convert columns containing dictionaries into JSON strings for better storage and processing.
columns_to_convert = ["attributes", "hours"]

for col in columns_to_convert:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)


In [8]:
df.drop_duplicates(inplace=True)
print(f"✅ After removing duplicates, {len(df)} rows remain")

✅ After removing duplicates, 178399 rows remain


In [9]:
df_sorted = df.sort_values(by="name")
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
69775,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}","Preschools, Education, Local Services, Child C...",Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,NaN,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
33991,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Beauty & Spas, Barbers",Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
144788,112 S Center St,"{""BusinessAcceptsCreditCards"": ""True"", ""Busine...","Barbers, Beauty & Spas",Plainfield,1,39.702890,-86.402923,Leland's Barbershop,46168,6,4.5,IN,NaN,"{""Friday"": ""7:0-17:0"", ""Saturday"": ""7:0-14:0"",..."
57020,8131 Kingston St,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Arts & Entertainment, Venues & Event Spaces, E...",Avon,0,39.759701,-86.378301,Xtreme Laser Tag Avon,46123,12,4.0,IN,f9pqgFRKMoN75uqzxd7KHw,"{""Friday"": ""17:0-23:0"", ""Saturday"": ""10:0-23:0..."
33505,14351 N Dale Mabry Hwy,"{""Ambience"": ""{'touristy': False, 'hipster': F...","Pizza, Breakfast & Brunch, Restaurants, Food, ...",Tampa,0,28.077936,-82.506174,"""Genuino"" Italian Cafe'",33618,60,4.5,FL,ihYl-uK0PYNPeFskcNAr-g,"{""Friday"": ""9:0-23:0"", ""Saturday"": ""9:0-23:0"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31365,900 Terry Pkwy,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Bakeries, Food, Desserts, Patisserie/Cake Shop",Terrytown,1,29.892304,-90.031530,éclair délicieux,70056,40,4.5,LA,2KlvBObysXe00p7phB45_A,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
85930,109 S 10th St,"{""Alcohol"": ""u'beer_and_wine'"", ""Ambience"": ""{...","Restaurants, Cafes, Food, Coffee & Tea",Boise,1,43.616756,-116.206361,ā café,83702,48,5.0,ID,VUKhLVz4X4zVS5wwjAxQjg,"{""Friday"": ""8:0-15:0"", ""Monday"": ""0:0-0:0"", ""S..."
17887,"943 E University Blvd, Ste 101",NaN,"Apartments, University Housing, Home Services,...",Tucson,1,32.232275,-110.957672,ōLiv Tucson,85719,11,1.0,AZ,NaN,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
129908,750 Ladera Ln,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Real Estate, Home Services, Churches, Fitness ...",Montecito,1,34.440557,-119.578843,​Silent Stay Meditation & Retreat Center,93108,33,5.0,CA,QB6O8M_lW3DSvXHhbylEjQ,"{""Friday"": ""9:0-17:0"", ""Monday"": ""9:0-17:0"", ""..."


In [10]:
df_sorted.isna().sum()

address             0
attributes      16262
categories        123
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     59621
hours           27498
dtype: int64

In [11]:
duplicates_missing_id = df_sorted[df_sorted.duplicated(subset=df_sorted.columns.difference(["business_id"]), keep=False)]
print(f"Found {len(duplicates_missing_id)} rows where everything is identical except for business_id.")


Found 56106 rows where everything is identical except for business_id.


In [13]:
# Grouping to keep only unique combinations while retaining business_id
df_filled_ids = df_sorted.dropna(subset=["business_id"]).drop_duplicates(subset=df_sorted.columns.difference(["business_id"]).tolist())



In [14]:
df_sorted = df_sorted.merge(df_filled_ids[["business_id"] + df_sorted.columns.difference(["business_id"]).tolist()],
                            on=df_sorted.columns.difference(["business_id"]).tolist(),
                            how="left",
                            suffixes=("", "_filled"))

# If business_id is missing, fill it with the matched one
df_sorted["business_id"] = df_sorted["business_id"].fillna(df_sorted["business_id_filled"])

# Remove the temporary column
df_sorted.drop(columns=["business_id_filled"], inplace=True)


In [15]:
df_sorted.isna().sum()

address             0
attributes      16262
categories        123
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     31568
hours           27498
dtype: int64

In [16]:
df_sorted.drop_duplicates(inplace=True)
df_sorted.isna().sum()

address             0
attributes      13744
categories        103
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     31568
hours           23223
dtype: int64

In [17]:
# Remove rows with missing business_id and duplicates based on key fields
df_filled_ids = df_sorted.dropna(subset=["business_id"]).drop_duplicates(subset=["name", "city", "address"])

# Merge with the cleaned dataset to fill missing business_id values
df_sorted = df_sorted.merge(df_filled_ids[["name", "city", "address", "business_id"]],
                            on=["name", "city", "address"],
                            how="left",
                            suffixes=("", "_filled"))

# Fill missing business_id values from matched records
df_sorted["business_id"] = df_sorted["business_id"].fillna(df_sorted["business_id_filled"])
df_sorted.drop(columns=["business_id_filled"], inplace=True)

# Check if any NaN values remain in business_id
print(f" Remaining {df_sorted['business_id'].isna().sum()} records with NaN in business_id after update.")


 Remaining 31534 records with NaN in business_id after update.


In [18]:
df_sorted["business_id"] = df_sorted["business_id"].fillna("Unknown_" + df_sorted.index.to_series().astype(str))

In [19]:
df_sorted.isna().sum()

address             0
attributes      13744
categories        103
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id         0
hours           23223
dtype: int64

In [20]:
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}","Preschools, Education, Local Services, Child C...",Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
1,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Beauty & Spas, Barbers",Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
2,112 S Center St,"{""BusinessAcceptsCreditCards"": ""True"", ""Busine...","Barbers, Beauty & Spas",Plainfield,1,39.702890,-86.402923,Leland's Barbershop,46168,6,4.5,IN,Unknown_2,"{""Friday"": ""7:0-17:0"", ""Saturday"": ""7:0-14:0"",..."
3,8131 Kingston St,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Arts & Entertainment, Venues & Event Spaces, E...",Avon,0,39.759701,-86.378301,Xtreme Laser Tag Avon,46123,12,4.0,IN,f9pqgFRKMoN75uqzxd7KHw,"{""Friday"": ""17:0-23:0"", ""Saturday"": ""10:0-23:0..."
4,14351 N Dale Mabry Hwy,"{""Ambience"": ""{'touristy': False, 'hipster': F...","Pizza, Breakfast & Brunch, Restaurants, Food, ...",Tampa,0,28.077936,-82.506174,"""Genuino"" Italian Cafe'",33618,60,4.5,FL,ihYl-uK0PYNPeFskcNAr-g,"{""Friday"": ""9:0-23:0"", ""Saturday"": ""9:0-23:0"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,900 Terry Pkwy,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Bakeries, Food, Desserts, Patisserie/Cake Shop",Terrytown,1,29.892304,-90.031530,éclair délicieux,70056,40,4.5,LA,2KlvBObysXe00p7phB45_A,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150342,109 S 10th St,"{""Alcohol"": ""u'beer_and_wine'"", ""Ambience"": ""{...","Restaurants, Cafes, Food, Coffee & Tea",Boise,1,43.616756,-116.206361,ā café,83702,48,5.0,ID,VUKhLVz4X4zVS5wwjAxQjg,"{""Friday"": ""8:0-15:0"", ""Monday"": ""0:0-0:0"", ""S..."
150343,"943 E University Blvd, Ste 101",NaN,"Apartments, University Housing, Home Services,...",Tucson,1,32.232275,-110.957672,ōLiv Tucson,85719,11,1.0,AZ,Unknown_150343,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150344,750 Ladera Ln,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Real Estate, Home Services, Churches, Fitness ...",Montecito,1,34.440557,-119.578843,​Silent Stay Meditation & Retreat Center,93108,33,5.0,CA,QB6O8M_lW3DSvXHhbylEjQ,"{""Friday"": ""9:0-17:0"", ""Monday"": ""9:0-17:0"", ""..."


In [22]:
import re
from thefuzz import process


# Step 2: Clean city names
def clean_city_name(city):
    """
    Cleans city names by:
    - Removing leading/trailing spaces
    - Replacing multiple spaces with a single space
    - Converting to Title Case
    - Removing dots and commas
    """
    if pd.isna(city):
        return None
    city = city.strip()
    city = re.sub(r"\s+", " ", city)
    city = city.title()
    city = re.sub(r"[.,]", "", city)
    return city

df_sorted["city"] = df_sorted["city"].apply(clean_city_name)

# Standardize city names using fuzzy matching
unique_cities = df_sorted["city"].dropna().unique()
city_mapping = {}

for city in unique_cities:
    result = process.extractOne(city, city_mapping.keys(), score_cutoff=80)

    if result:
        best_match, score = result
        city_mapping[city] = best_match  # Assign the most appropriate name
    else:
        city_mapping[city] = city   #Keep original name if no match found

df_sorted["city"] = df_sorted["city"].map(city_mapping)

print(f"Total unique cities after processing: {df_sorted['city'].nunique()}")


Total unique cities after processing: 755


In [23]:
for city in sorted(df_sorted["city"].unique()):
    print(city)

Abington
Afton
Alberta Park Industrial
Aldan
Algiers
Aliso Viejo
Alloway
Almonesson
Alton
Ambler
Andalusia
Antioch
Apollo Beach
Apopka
Arabi
Ardmore
Arizona
Arnold
Ashland
Ashland City
Aston
Atco
Audubon
Austin
Avon
Bala Cynwyd
Ballwin
Balm
Bargersville
Barnhart
Barto
Bayonet Point
Bear
Beaumont
Beech Grove
Bel Ridge
Belle Chasse
Belle Meade
Belleair
Belleair Bluffs
Bellefontaine
Bellefonte
Belleville
Bellevue
Bellmawr
Bellville
Belmont Hills
Bennington
Bensalem
Berkeley
Berlin
Berwyn
Bethel Township
Beverly
Birchrunville
Black Jack
Blackwood
Blooming Glen
Blue Bell
Blvd
Boise
Boone
Boothwyn
Bordentown
Boston
Boulevard
Boutte
Boyertown
Bradenton
Brandon
Breckenridge Hills
Brentwood
Brentwood - Cool Springs
Bridge City
Bridgeport
Bridgeton
Bristol
Brookhaven
Brooklawn
Brooksville
Broomall
Brownsburg
Bryn Athyn
Bryn Mawr
Buckingham
Bucks
Burlington
Bywater
Cahokia
Caln
Camby
Camden
Cane Ridge
Carmel
Carneys Point
Carpinteria
Carrollwood
Casas Adobes
Caseyville
Catalina
Cedar Brook
Cedars

In [24]:
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}","Preschools, Education, Local Services, Child C...",Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
1,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Beauty & Spas, Barbers",Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
2,112 S Center St,"{""BusinessAcceptsCreditCards"": ""True"", ""Busine...","Barbers, Beauty & Spas",Plainfield,1,39.702890,-86.402923,Leland's Barbershop,46168,6,4.5,IN,Unknown_2,"{""Friday"": ""7:0-17:0"", ""Saturday"": ""7:0-14:0"",..."
3,8131 Kingston St,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Arts & Entertainment, Venues & Event Spaces, E...",Avon,0,39.759701,-86.378301,Xtreme Laser Tag Avon,46123,12,4.0,IN,f9pqgFRKMoN75uqzxd7KHw,"{""Friday"": ""17:0-23:0"", ""Saturday"": ""10:0-23:0..."
4,14351 N Dale Mabry Hwy,"{""Ambience"": ""{'touristy': False, 'hipster': F...","Pizza, Breakfast & Brunch, Restaurants, Food, ...",Tampa,0,28.077936,-82.506174,"""Genuino"" Italian Cafe'",33618,60,4.5,FL,ihYl-uK0PYNPeFskcNAr-g,"{""Friday"": ""9:0-23:0"", ""Saturday"": ""9:0-23:0"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,900 Terry Pkwy,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Bakeries, Food, Desserts, Patisserie/Cake Shop",Terrytown,1,29.892304,-90.031530,éclair délicieux,70056,40,4.5,LA,2KlvBObysXe00p7phB45_A,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150342,109 S 10th St,"{""Alcohol"": ""u'beer_and_wine'"", ""Ambience"": ""{...","Restaurants, Cafes, Food, Coffee & Tea",Boise,1,43.616756,-116.206361,ā café,83702,48,5.0,ID,VUKhLVz4X4zVS5wwjAxQjg,"{""Friday"": ""8:0-15:0"", ""Monday"": ""0:0-0:0"", ""S..."
150343,"943 E University Blvd, Ste 101",NaN,"Apartments, University Housing, Home Services,...",Tucson,1,32.232275,-110.957672,ōLiv Tucson,85719,11,1.0,AZ,Unknown_150343,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150344,750 Ladera Ln,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Real Estate, Home Services, Churches, Fitness ...",Montecito,1,34.440557,-119.578843,​Silent Stay Meditation & Retreat Center,93108,33,5.0,CA,QB6O8M_lW3DSvXHhbylEjQ,"{""Friday"": ""9:0-17:0"", ""Monday"": ""9:0-17:0"", ""..."


In [25]:
# Split categories by comma and expand them into separate rows
df_exploded = df_sorted.assign(categories=df_sorted["categories"].str.split(",")).explode("categories")

# Remove extra spaces and convert to Title Case
df_exploded["categories"] = df_exploded["categories"].str.strip().str.title()


print(f"Total unique categories after processing: {df_exploded['categories'].nunique()}")

df_sorted = df_exploded

Total unique categories after processing: 1311


In [26]:
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Preschools,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Education,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Local Services,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Child Care & Day Care,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
1,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...",Beauty & Spas,Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Massage,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Naturopathic/Holistic,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Health & Medical,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Traditional Chinese Medicine,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
